In [ ]:
import requests
import pandas as pd
import os
from tqdm.auto import tqdm
import json
import math
import numpy as np

from src.textprocessing.preprocess import normalize_text, remove_punctuation

### Extracción

In [ ]:
VERSION_MODEL_0 = "0.8.0"
VERSION_DATA_0 = "001"

In [ ]:
df = pd.read_csv("../data/cleaned/latam_cleaned_data_2023-10-31.csv")
df = df.sample(frac = 0.1).reset_index(drop = True)

df[["salary_min_form", "salary_max_form", "currency_form", "time_lapse_form"]] = ""

In [ ]:
def extract_salary(description, job_id, location):
    response = requests.post('http://localhost:3000/predict',
                             data = json.dumps({"description" : description,
                                                "jobid" : job_id,
                                                "location" : location
                                               }),
                             headers = {"Content-Type" : "application/json"})

    if response.status_code == 200:
        return response.json()
    
    return {'max': 0, 'min': 0, 'error_status_code': response.status_code}

In [ ]:
salary_info = list()
total_tokens = 0
accumulate_cost = 0
progress_steps = math.ceil(df.shape[0]*0.15)

for index, row in tqdm(df.iterrows(), total = df.shape[0]):
    
    try:
        
        salary_output = extract_salary(description = row.description, job_id = row.job_id, location = row.country)
        sent_tokens = salary_output["token_count_sent_chat_gpt"]
        total_tokens += sent_tokens

        cost = float(salary_output["cost"])
        accumulate_cost += cost

        orignal_description_tokens = len(row.description.split(" "))
        reduce_description_len = len(salary_output["reduce_description"].split(" "))
        ratio_sent_original_description = reduce_description_len/orignal_description_tokens

        salary_info.append((row.job_id, salary_output["reduce_description"], salary_output["min"], salary_output["max"], salary_output["currency"], salary_output["time_lapse"], salary_output["source"], sent_tokens, orignal_description_tokens, ratio_sent_original_description, cost))

        if (index%progress_steps) == 0:
            
            print(f"[output salary: {salary_output}]")
            print("[------- Full Description -------]")
            print(row.description)
            print("[------- Reduce description -------]")
            print(salary_output["reduce_description"])
            print(f"==== Tokens sent for this JD:{sent_tokens} - Total sent tokens: {total_tokens} - tokens in complete description:{orignal_description_tokens} = Ratio sent/original description so far:{ratio_sent_original_description} | Cost for this JD:{cost} - Total cost so far: {accumulate_cost} ====")

    except Exception as e:
        
        print(f"error: {e}")

print(f"estimated_cost: {accumulate_cost} per: {df.shape[0]} jobs description | Total sent tokens: {total_tokens}")

In [ ]:
salary_df = pd.DataFrame(salary_info,
                         columns=[
                                    "job_id",
                                    "reduce_description",
                                    f"salary_min_gpt_{VERSION_MODEL_0}",
                                    f"salary_max_gpt_{VERSION_MODEL_0}",
                                    f"currency_gpt_{VERSION_MODEL_0}",
                                    f"time_lapse_gpt_{VERSION_MODEL_0}",
                                    f"source_{VERSION_MODEL_0}",
                                    f"token_count_sent_chat_gpt_{VERSION_MODEL_0}",
                                    "token_count_original_description",
                                    "token_rate_sent_original_description",
                                    f"gpt_cost_{VERSION_MODEL_0}"
                                ])

#### Agregamos faltantes

In [ ]:
jobs_with_salary = pd.merge(df, salary_df, on = "job_id", how = "left")

#### ¿Cuántas ofertas de empleo con salarios tenemos?

In [ ]:
all_jobs_count = jobs_with_salary.shape[0]
jobs_with_salary_count = jobs_with_salary[jobs_with_salary[f"salary_min_gpt_{VERSION_MODEL_0}"] > 0].shape[0]

print(f"Salary jobs count: {all_jobs_count} | {jobs_with_salary_count} - jobs with salary {round(jobs_with_salary_count/all_jobs_count, 3)}%")

In [ ]:
all_jobs_by_country = df.groupby(by = ["country"]).size().sort_values(ascending=False).reset_index(name = "count_all")

all_jobs_by_country

In [ ]:
# jobs_with_salary = jobs_with_salary[["job_id", "country", "experience", "experience_levels", "description", "email",
#                                      "Especialidad", "Perfil", "remote_work", "tech_skills", "title", "company_name",
#                                      "location", "source", "date_posted", "contract_type", "salary_min_gpt_0.8.0",
#                                      "salary_max_gpt_0.8.0", "currency_gpt_0.8.0", "time_lapse_gpt_0.8.0"]]

# jobs_with_salary = jobs_with_salary.rename(columns = {"salary_min_gpt_0.8.0" : "salary_min",
#                                                       "salary_max_gpt_0.8.0" : "salary_max",
#                                                       "currency_gpt_0.8.0" : "currency",
#                                                       "time_lapse_gpt_0.8.0" : "time_lapse"})

# jobs_with_salary["currency"].replace(" ", np.nan, inplace = True)
# jobs_with_salary["time_lapse"].replace(" ", np.nan, inplace = True)

jobs_with_salary

In [ ]:
jobs_with_salary.to_csv("../data/with_salaries/latam_cleaned_data_2023-10-31_with_salaries.csv", index = False, sep = ",")

### Limpieza

In [ ]:
# Set up chatgpt model
# My token

with open(file = "openai_key.txt", mode = "r") as file:
    os.environ['CHAT_GPT_API_KEY'] = file.read()

from src.api import process_request

NGRAMS = 5
OVERLAP = 0.3

In [ ]:
def create_job_rerun_id(row):
    return "rerun_" + row.job_id

In [ ]:
EXPORT_NAME = "spain_000"
HEADERS = ["reduce_description", 
           f"salary_min_gpt_{VERSION_MODEL_0}", 
           f"salary_max_gpt_{VERSION_MODEL_0}", 
           f"currency_gpt_{VERSION_MODEL_0}", 
           f"time_lapse_gpt_{VERSION_MODEL_0}"]

# Para concatenar
discarted_rows_list = list()

In [ ]:
processed_df = pd.read_csv("../data/with_salaries/spain_cleaned_data_2023-10-31_with_salaries.csv")

processed_df = processed_df.dropna(subset = "job_id")
processed_df = processed_df.dropna(subset = "description")
processed_df["description_normalized"] = processed_df.description.apply(normalize_text)
processed_df["description_normalized"] = processed_df.description.apply(remove_punctuation)
processed_df = processed_df.drop_duplicates(subset=["description_normalized"])
print(f"Preprocessed jobs: ", processed_df.shape[0])

OG_SIZE = processed_df.shape[0]

In [ ]:
processed_df[processed_df[F"salary_min_gpt_{VERSION_MODEL_0}"] != 0].shape

In [ ]:
INIT_PERCENTAJE_JOBS_WITH_SALARIES = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] != 0].shape[0] / processed_df.shape[0]

In [ ]:
processed_df.columns

In [ ]:
processed_df[f"currency_gpt_{VERSION_MODEL_0}"].unique()

#### El modelo entiende currency como una unidad de cantidad

In [ ]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "UTC"].head()[HEADERS]

In [ ]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "REQ"].head()[HEADERS]

In [ ]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "Percentage"].head()[HEADERS]

In [ ]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "Salarios mínimos"].head()[HEADERS]

In [ ]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "PLN"].head()[HEADERS]

In [ ]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "preferred"].head()[HEADERS]

In [ ]:
processed_df[processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "Unknown"].head()[HEADERS]

#### currencies

In [ ]:
# El primer filtro deberían ser con las currencies que representan dinero
allowed_currencies = ['MXN', 'CLP', 'USD', 'COP', 'MVC', 'DOP', 'EUR', 'GTQ', 'CRC', 'KRW', 'SVC', 'HNL', 'UYU', 'PEN',
                     'BRL', 'ARS', 'NTQ', 'HRS', 'QAR', 'GBP', 'PHP', 'CAD', 'JMD', 'BMD', 'MMK',
                     'ZAR', 'NIO', 'PAB', 'PYG', 'MYR', 'PLN', 'AED', 'INR', 'CST', 'CHF', 'BOB']

In [ ]:
processed_df = processed_df[processed_df[f"currency_gpt_{VERSION_MODEL_0}"].isin(allowed_currencies)]

print(processed_df.shape)

In [ ]:
def create_discarted_df(discarted_rows_list):
    return pd.concat([pd.DataFrame(df) for df in discarted_rows_list])

def get_discarted_sample(discarted_df, HEADERS):
    return discarted_df[discarted_df[f"salary_min_gpt_{VERSION_MODEL_0}"] != 0].sample(n = 10)[HEADERS]

In [ ]:
# Ejemplo para visualizar
processed_df.sample(n = 0)[["reduce_description", f"salary_min_gpt_{VERSION_MODEL_0}", f"salary_max_gpt_{VERSION_MODEL_0}", f"currency_gpt_{VERSION_MODEL_0}", f"time_lapse_gpt_{VERSION_MODEL_0}"]]

#### time_lapse

In [ ]:
# El segundo filtro sería time_lapse para reducir alguna incoherencia
processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"].unique()

In [ ]:
valid_time_lapses = ['month','year', 'hour', 'annual']

In [ ]:
discarted_rows_list.append(processed_df[~processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"].isin(valid_time_lapses)])

processed_df = processed_df[processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"].isin(valid_time_lapses)]

print(processed_df.shape)

In [ ]:
# Ejemplo para visualizar
processed_df.sample(n = 10)[HEADERS]

#### General

In [ ]:
processed_df = processed_df.astype({f"salary_min_gpt_{VERSION_MODEL_0}": "float64", f"salary_max_gpt_{VERSION_MODEL_0}":"float64"})

In [ ]:
processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] > 0]

print(processed_df.shape)

In [ ]:
# 40 o 45 horas a la semana
for week_hours in [25, 40, 45]:
    week_hours = processed_df[(processed_df["reduce_description"].str.contains(f"{week_hours} hours"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] == week_hours)|(processed_df[f"salary_max_gpt_{VERSION_MODEL_0}"] == week_hours))]
    processed_df = processed_df[~processed_df.job_id.isin(week_hours.job_id)]

# años
for age in range(50):
    years_context = processed_df[(processed_df["reduce_description"].str.contains(f"{age} years"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] == age)|(processed_df[f"salary_max_gpt_{VERSION_MODEL_0}"] == age))]
    processed_df = processed_df[~processed_df.job_id.isin(years_context.job_id)]

for age in range(5000):
    years_context = processed_df[(processed_df["reduce_description"].str.contains(f"{age} fastest"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] == age)|(processed_df[f"salary_max_gpt_{VERSION_MODEL_0}"] == age))]
    processed_df = processed_df[~processed_df.job_id.isin(years_context.job_id)]

#### Números que representan ganancias u otras cosas

In [ ]:
processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] >= 1000000000].head()[HEADERS]

In [ ]:
processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] < 1000000000]

print(processed_df.shape)

In [ ]:
try:
    processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] >= 50000000].sample(n = 10)[HEADERS]
    
except:
    pass

In [ ]:
processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] < 50000000]

print(processed_df.shape)

In [ ]:
processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] < 10000000]

print(processed_df.shape)

In [ ]:
# Ejemplo para visualizar
processed_df.sample(n = 10)[HEADERS + ["location"]]

#### Filtros finales

In [ ]:
# Si contiene "osde 210", "osde 310", "osde 410"
osde_df = processed_df[(processed_df["reduce_description"].str.contains("osde")) & ((processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] == 310)|(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] == 210)|(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] == 410))]

In [ ]:
osde_df[HEADERS + ["location"]]

In [ ]:
processed_df = processed_df[~processed_df.job_id.isin(osde_df.job_id)]

In [ ]:
# Ejemplo para visualizar
processed_df[processed_df["reduce_description"].str.contains("osde")][HEADERS + ["location"]]

In [ ]:
exclude_jobs_id = processed_df[(processed_df["reduce_description"].str.contains("6 months to months to 1 year")) & (processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] == 6) & (processed_df[f"salary_max_gpt_{VERSION_MODEL_0}"] == 12)].job_id

In [ ]:
exclude_jobs_list = list()

for range_min, range_max in [(9, 18), (9, 5), (3, 5), (6, 1), (6, 12), (3, 4)]:
    exclude_jobs_list.append(processed_df[(((processed_df["reduce_description"].str.contains(str(range_min))) & (processed_df["reduce_description"].str.contains(str(range_max))))
                                           ) & (processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] == range_min) & (processed_df[f"salary_max_gpt_{VERSION_MODEL_0}"] == range_max)])

In [ ]:
# Añade todos los trabajos excluidos, OSDE y rango horario
exclude_jobs_df = pd.concat(exclude_jobs_list)

discarted_rows_list.append(exclude_jobs_list)

In [ ]:
exclude_jobs_df[HEADERS+["location"]]

In [ ]:
processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

##### Salarios de 1$ mensual o anual

In [ ]:
try:
    processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] <= 1) & (processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] != "hour")].sample(n = 10)[HEADERS+["location"]]

except:
    pass

In [ ]:
exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] <= 1) & (processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"] != "hour")]

In [ ]:
processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

In [ ]:
# Eliminamos salario mensual superior a 50K USD
exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] > 50000) & (processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"] == "month") & ((processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "USD") | (processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "CAD") | (processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "EUR"))]

discarted_rows_list.append(exclude)

In [ ]:
processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

In [ ]:
# Excluimos horas extra USD
exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] > 200) & (processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"] == "hour") & ((processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "USD") | (processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "CAD") | (processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "EUR"))]

discarted_rows_list.append(exclude)

In [ ]:
discarted_row_df = create_discarted_df(discarted_rows_list = discarted_rows_list)

In [ ]:
discarted_row_df[pd.isna(discarted_row_df[f"time_lapse_gpt_{VERSION_MODEL_0}"])][HEADERS].sample(n = 10)

In [ ]:
# Excluimos salarios muy bajos
exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] < 500) & (processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"] == "month") & ((processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "USD") | (processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "CAD") | (processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "EUR"))]
processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] < 10000) & (processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"] == "year") & ((processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "USD") | (processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "CAD") | (processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "EUR"))]
processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] < 3) & (processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"] == "hour") & ((processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "USD") | (processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "CAD") | (processed_df[f"currency_gpt_{VERSION_MODEL_0}"] == "EUR"))]
processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

##### Mantenemos solo timelapses con NaN

In [ ]:
discarted_row_df = discarted_row_df[pd.isna(discarted_row_df[f"time_lapse_gpt_{VERSION_MODEL_0}"])]

In [ ]:
get_discarted_sample(discarted_df = discarted_row_df, HEADERS = HEADERS)

In [ ]:
# Testeamos que puede recuperarse con la nueva versión del modelo
reprocessed_records_list = list()

discarted_row_df["location"] = discarted_row_df["location"].fillna(" ")
discarted_row_df= discarted_row_df.dropna(subset = "job_id")
total_cost = 0

for _, row in tqdm(discarted_row_df.iterrows(), total=discarted_row_df.shape[0]):
    salary_output = process_request(description = row.description, location = row.location, jobid = create_job_rerun_id(row), ngrams = NGRAMS, overlap_limit = OVERLAP)

    sent_tokens = salary_output["token_count_sent_chat_gpt"]

    cost = float(salary_output["cost"])
    total_cost += cost
    orignal_description_tokens = len(row.description.split(" "))
    reduce_description_len = len(salary_output["reduce_description"].split(" "))
    ratio_sent_original_description = reduce_description_len/orignal_description_tokens

    reprocessed_records_list.append((row.job_id, salary_output["reduce_description"], salary_output["min"], salary_output["max"], salary_output["currency"], salary_output["time_lapse"], salary_output["source"], sent_tokens, orignal_description_tokens, ratio_sent_original_description, cost))

print(f"TOTAL COST OF REPROCESSING: {total_cost}")

In [ ]:
reprocessed_records_list[0]

In [ ]:
reprocessed_records_df = pd.DataFrame(reprocessed_records_list,
                         columns=[
                                    "job_id",
                                    "reduce_description",
                                    f"salary_min_gpt_{VERSION_MODEL_0}",
                                    f"salary_max_gpt_{VERSION_MODEL_0}",
                                    f"currency_gpt_{VERSION_MODEL_0}",
                                    f"time_lapse_gpt_{VERSION_MODEL_0}",
                                    f"source_{VERSION_MODEL_0}",
                                    f"token_count_sent_chat_gpt_{VERSION_MODEL_0}",
                                    "token_count_original_description",
                                    "token_rate_sent_original_description",
                                    f"gpt_cost_{VERSION_MODEL_0}"
                                ]
            )

In [ ]:
get_discarted_sample(discarted_df = reprocessed_records_df, HEADERS = HEADERS)

#### Todos los filtros

In [ ]:
def filter_rows(processed_df):
    
    processed_df = processed_df[processed_df[f"currency_gpt_{VERSION_MODEL_0}"].isin(allowed_currencies)]

    processed_df = processed_df[processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"].isin(valid_time_lapses)]

    processed_df = processed_df.astype({f"salary_min_gpt_{VERSION_MODEL_0}": "float64", f"salary_max_gpt_{VERSION_MODEL_0}":"float64"})

    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]>0]

    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]>1]

    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]<1000000000]

    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]<50000000]

    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]<20000000]

    processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]<10000000]

    osde_df = processed_df[(processed_df["reduce_description"].str.contains("osde"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]==310)|(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]==210)|(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]==410))]


    processed_df = processed_df[~processed_df.job_id.isin(osde_df.job_id)]

    for range_min, range_max in [(9, 18), (9, 5), (3, 5), (6, 1), (6, 12), (3, 4)]:
        exclude_jobs_list.append(processed_df[(((processed_df["reduce_description"].str.contains(str(range_min)))&(processed_df["reduce_description"].str.contains(str(range_max))))
                                            )&(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]==range_min)&(processed_df[f"salary_max_gpt_{VERSION_MODEL_0}"]==range_max)])
    exclude_jobs_df = pd.concat(exclude_jobs_list)

    processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

    exclude_jobs_df = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]<=1)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"]!="hour")]
    processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

    exclude_jobs_df = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]>50000)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"]=="month")&(processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="USD")]
    processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

    exclude_jobs_df = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]>200)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"]=="hour")&(processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="USD")]
    processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

    # Demasiado bajo para ser un salario
    exclude_jobs_df = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]<2000)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"]=="hour")&(processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="USD")]
    processed_df = processed_df[~processed_df.job_id.isin(exclude_jobs_df.job_id)]

    exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]<500)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"]=="month")&((processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="USD")|(processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="CAD")|(processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="EUR"))]
    processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

    exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]<10000)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"]=="year")&((processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="USD")|(processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="CAD")|(processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="EUR"))]
    processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

    exclude = processed_df[(processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]<3)&(processed_df[f"time_lapse_gpt_{VERSION_MODEL_0}"]=="hour")&((processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="USD")|(processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="CAD")|(processed_df[f"currency_gpt_{VERSION_MODEL_0}"]=="EUR"))]
    processed_df = processed_df[~processed_df.job_id.isin(exclude.job_id)]

    # 40 o 45 horas a la semana
    for week_hours in [25, 40, 45]:
        week_hours = processed_df[(processed_df["reduce_description"].str.contains(f"{week_hours} hours"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]==week_hours)|(processed_df[f"salary_max_gpt_{VERSION_MODEL_0}"]==week_hours))]
        processed_df = processed_df[~processed_df.job_id.isin(week_hours.job_id)]

    # años
    for age in range(50):
        years_context = processed_df[(processed_df["reduce_description"].str.contains(f"{age} years"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]==age)|(processed_df[f"salary_max_gpt_{VERSION_MODEL_0}"]==age))]
        processed_df = processed_df[~processed_df.job_id.isin(years_context.job_id)]

    for age in range(5000):
        years_context = processed_df[(processed_df["reduce_description"].str.contains(f"{age} fastest"))&((processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"]==age)|(processed_df[f"salary_max_gpt_{VERSION_MODEL_0}"]==age))]
        processed_df = processed_df[~processed_df.job_id.isin(years_context.job_id)]

    return processed_df

In [ ]:
filtered_reprocessed_records_df = filter_rows(reprocessed_records_df)
processed_df = filter_rows(processed_df)

In [ ]:
try:
    get_discarted_sample(discarted_df = filtered_reprocessed_records_df, HEADERS = HEADERS)

except:
    pass

In [ ]:
processed_df = processed_df[processed_df[f"salary_min_gpt_{VERSION_MODEL_0}"] <= processed_df[f"salary_max_gpt_{VERSION_MODEL_0}"]]
filtered_reprocessed_records_df = filtered_reprocessed_records_df[filtered_reprocessed_records_df[f"salary_min_gpt_{VERSION_MODEL_0}"] <= filtered_reprocessed_records_df[f"salary_max_gpt_{VERSION_MODEL_0}"]]

In [ ]:
preprocessed_count = processed_df.shape[0]
rescued_count = filtered_reprocessed_records_df.shape[0]

print(f"Preprocessed: {preprocessed_count}")
print(f"Rescued: {rescued_count}")
print(f"% recovered: {rescued_count/preprocessed_count}")
print(f"Salary %: {preprocessed_count/OG_SIZE}")
print(f"Salary % with recovered: {(preprocessed_count+rescued_count)/OG_SIZE}")

In [ ]:
# Lo unimos al dataframe previamente preprocesado
processed_df = pd.concat([processed_df, filtered_reprocessed_records_df])

In [ ]:
get_discarted_sample(discarted_df = processed_df, HEADERS = HEADERS)

In [ ]:
processed_df.to_csv("../data/with_salaries/spain_cleaned_data_2023-10-31_with_salaries.csv", index = False, sep = ",")

In [ ]:
################################################################################################################################